# QC and mut rate for Illumina data

In [ ]:
#plotly libraries
import plotly.express as px
import plotly.colors as pc
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

import sklearn.metrics as metrics
import pandas as pd
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

#default renderer (svg means very small file size, visibility on github, high quality, but requires sometimes setting height and width)
pio.renderers.default = "svg"

colors = ["#363b3d", "#727b76", "#31a240", "#f93939", "#f79118", "#de7b06", "#9b308f", "#dc759b"]
#additional defaults
px.defaults.color_discrete_sequence = ["rgb(100,100,100)"] + pc.qualitative.D3
px.defaults.width=1000
px.defaults.height=800

#try setting this as default for histograms
#fig.update_traces(marker_line_width=0.2)

#set default template as "simple_white" (no background, no grid lines)
pio.templates["simple_white"].layout["yaxis"]["showgrid"] = True
pio.templates.default = "simple_white"

colorscale = pc.sequential.Plasma
print(colorscale)
colorscale = [colorscale[0]] + colorscale[3:8]
colorscale

In [ ]:
data_folder = "..."


In [ ]:
import pandas as pd
samplesheet = pd.read_excel("./samplesheet.xlsx", engine="openpyxl")
samplesheet

In [ ]:
samples = samplesheet["Sample"].values
samples

In [ ]:
in_dir = f"{data_folder}/fastq"
out_dir = f"{data_folder}/fastq_stats"

os.makedirs(f"{out_dir}", exist_ok=True)

num_threads = 1
slurm = Slurm("stats", {"partition" : "cpu", "mem" : "10G", "cpus-per-task" : num_threads, "time" : "30","mail-type" : "FAIL,INVALID_DEPEND", "mail-user" : "patrick.bohn@helmholtz-hiri.de"})

for sample in samples:
    
    commands = []
    for read in ["_1", "_2"]:
        for stat in ["position_hist"]:
            fastq_infile = f"{in_dir}/{sample}{read}.fq.gz"
            outfile = f"{out_dir}/{sample}{read}"
            command = f"python3 functions/calc_per_read.py -i {fastq_infile} -o {outfile} -s {stat}"
            commands.append(command)
    #slurm.run(command)
    slurm.run("\n".join(commands))

    

## Calculate per read statistics for read1 and read2

In [ ]:
num_quality_scores = 40
max_read_length = 150

per_pos_hist_r1 = np.zeros((num_quality_scores,max_read_length), dtype=int)
per_pos_hist_r2 = np.zeros((num_quality_scores,max_read_length), dtype=int)

for sample in samples:#samples
    
    read = "_1"
    outfile = f"{out_dir}/{sample}{read}_read_quality_per_position_histogram.csv"
    tmp_per_pos_hist_r1 = np.genfromtxt(outfile)
    
    per_pos_hist_r1 = per_pos_hist_r1 + tmp_per_pos_hist_r1
    
    read = "_2"
    outfile = f"{out_dir}/{sample}{read}_read_quality_per_position_histogram.csv"
    tmp_per_pos_hist_r2 = np.genfromtxt(outfile)
    
    per_pos_hist_r2 = per_pos_hist_r2 + tmp_per_pos_hist_r2
    
    

In [ ]:
#assumes histogram is 0-40
def calc_median_qscore_from_hist(histogram):
    total_n = np.sum(histogram)
    
    half_n = total_n/2
    
    for qscore in np.arange(0,40):
        cum_n = np.sum(histogram[:qscore])
        if cum_n > half_n:
            return qscore -1

def plot_per_pos_median_mean_cov(per_pos_hist):
    coverage_per_pos = np.sum(per_pos_hist, axis=0)

    qscores = np.arange(1,41)
    error_prob_per_qscore = 10**(-qscores/10)

    error_per_pos = per_pos_hist.T * error_prob_per_qscore
    sum_error_per_pos = np.sum(error_per_pos, axis= 1)
    mean_error_per_pos = sum_error_per_pos/coverage_per_pos
    mean_qscore_per_pos = np.log10(mean_error_per_pos)*-10
    
    overall_error_prob = np.sum(error_per_pos) / np.sum(coverage_per_pos)
    
    overall_mean_qscore = np.log10(overall_error_prob)*-10
    print("Overall mean error rate:", overall_error_prob*100, "%")
    print("Overall mean qscore:", overall_mean_qscore)
    
    median_qscore_per_pos = np.apply_along_axis(calc_median_qscore_from_hist, 0, per_pos_hist)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    positions = np.arange(per_pos_hist.shape[1])
    fig.add_trace(go.Scattergl(x=positions, y=coverage_per_pos, name="coverage", line_color="grey"), secondary_y=True)
    fig.add_trace(go.Scattergl(x=positions, y=mean_qscore_per_pos, name="mean_qscore", line_color="green"))
    fig.add_trace(go.Scattergl(x=positions, y=median_qscore_per_pos, name="median_qscore", line_color="blue"))
    
    fig.update_yaxes(rangemode="tozero", showgrid=False, secondary_y=True)
    fig.update_yaxes(range=[10,40], dtick=2, secondary_y=False)
    
    fig.show()
    return fig

# Plot per read statistics

In [ ]:
plot_per_pos_median_mean_cov(per_pos_hist_r1)

In [ ]:
plot_per_pos_median_mean_cov(per_pos_hist_r2)

In [ ]:
plot_per_pos_median_mean_cov(per_pos_hist_r1 + per_pos_hist_r2).write_image("figures/illumina_per_position_qscore.svg")

In [ ]:
out_dir = f"{data_folder}/fastq_stats"

read_median_histogram_r1 = np.array(40, dtype=int)
read_median_histogram_r2 = np.array(40, dtype=int)
for sample in samples:
    

    outfile = f"{out_dir}/{sample}_1_read_median_histogram.csv"
    hist = np.genfromtxt(outfile, delimiter=" ", dtype=int)
    read_median_histogram_r1 = read_median_histogram_r1 + hist[0]
    
    outfile = f"{out_dir}/{sample}_2_read_median_histogram.csv"
    hist = np.genfromtxt(outfile, delimiter=" ", dtype=int)
    read_median_histogram_r2 = read_median_histogram_r2 + hist[0]

fig = go.Figure()
fig.add_trace(go.Bar(y=hist[1], x=read_median_histogram_r1, name="r1", marker_color="darkblue",
            orientation='h'))
fig.add_trace(go.Bar(y=hist[1], x=read_median_histogram_r2, name="r2", marker_color="blue",
            orientation='h'))
fig.update_layout(title="Read median qscore histogram", width=300, height=500)
fig.update_yaxes(range = [10,40], dtick=1)
fig.write_image("figures/illumina_median_qscore_per_read_distribution.svg")
fig.show()

In [ ]:
out_dir = f"{data_folder}/fastq_stats"

read_median_histogram_r1 = np.array(40, dtype=int)
read_median_histogram_r2 = np.array(40, dtype=int)
for sample in samples:
    

    outfile = f"{out_dir}/{sample}_1_read_mean_histogram.csv"
    hist = np.genfromtxt(outfile, delimiter=" ")
    read_median_histogram_r1 = read_median_histogram_r1 + hist[0]
    
    outfile = f"{out_dir}/{sample}_2_read_mean_histogram.csv"
    hist = np.genfromtxt(outfile, delimiter=" ")
    read_median_histogram_r2 = read_median_histogram_r2 + hist[0]

fig = go.Figure()
fig.add_trace(go.Bar(y=hist[1], x=read_median_histogram_r1, name="r1", marker_color="green",
            orientation='h'))
fig.add_trace(go.Bar(y=hist[1], x=read_median_histogram_r2, name="r2", marker_color="darkgreen",
            orientation='h'))
fig.update_layout(title="Read mean qscore histogram", width=300, height=500)
fig.update_yaxes(range = [10,40], dtick=1)
fig.write_image("figures/illumina_mean_qscore_per_read_distribution.svg")
fig.show()

In [ ]:
def gen_cumulative_hist(histogram, norm=None):
    num_bins = len(histogram)
    cum_histogram = np.zeros(num_bins)
    
    for bin_index in np.arange(num_bins):
        cum_histogram[bin_index] = np.sum(histogram[:bin_index+1])
        
    if norm=="percent":
        return 100* cum_histogram/np.sum(histogram)
    else:
        return cum_histogram

In [ ]:
out_dir = f"{data_folder}/fastq_stats"

read_median_histogram_r1 = np.array(40, dtype=int)
read_median_histogram_r2 = np.array(40, dtype=int)
for sample in samples:
    

    outfile = f"{out_dir}/{sample}_1_read_mean_histogram.csv"
    hist = np.genfromtxt(outfile, delimiter=" ")
    read_median_histogram_r1 = read_median_histogram_r1 + hist[0]
    
    outfile = f"{out_dir}/{sample}_2_read_mean_histogram.csv"
    hist = np.genfromtxt(outfile, delimiter=" ")
    read_median_histogram_r2 = read_median_histogram_r2 + hist[0]

cum_read_mean_hist_r1 = gen_cumulative_hist(read_median_histogram_r1, norm="percent")
cum_read_mean_hist_r2 = gen_cumulative_hist(read_median_histogram_r2, norm="percent")

print(cum_read_mean_hist_r1)
fig = go.Figure()
fig.add_trace(go.Bar(x=hist[1], y=cum_read_mean_hist_r1, name="r1", marker_color="darkblue"))
fig.add_trace(go.Bar(x=hist[1], y=cum_read_mean_hist_r2, name="r2", marker_color="darkgreen"))
fig.update_layout(title="Cumulative read mean qscore histogram", width=500, height=400)
fig.update_yaxes(dtick=10)
fig.update_xaxes(range=[10,40], dtick=1)
fig.show()

fig = go.Figure()
fig.add_trace(go.Bar(x=hist[1], y=100-cum_read_mean_hist_r1, name="r1", marker_color="darkblue"))
fig.add_trace(go.Bar(x=hist[1], y=100-cum_read_mean_hist_r2, name="r2", marker_color="darkgreen"))
fig.update_layout(title="Inverse cumulative read mean qscore histogram", width=600, height=400)
fig.update_yaxes(dtick=10)
fig.update_xaxes(range=[10,40], dtick=1)
fig.show()

In [150]:
fqo = f"{data_folder}/fastq_minq{minq}/{sample}"
read1_mean_fastq = np.genfromtxt(f"{fqo}_mean_qscores_r1.csv")
read2_mean_fastq = np.genfromtxt(f"{fqo}_mean_qscores_r2.csv")
histogram_r1 = np.histogram(read1_mean_fastq, range=[0,40], bins=40)
histogram_r2 = np.histogram(read2_mean_fastq, range=[0,40], bins=40)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=histogram_r1[1], y=histogram_r1[0]))
fig.add_trace(go.Bar(x=histogram_r2[1], y=histogram_r2[0]))

# (optional) filter reads for minimal mean fastq score

In [ ]:
os.makedirs(f"{data_folder}/{outdir}", exist_ok=True)

num_threads = 1
slurm = Slurm("filter_fq", {"partition" : "cpu", "mem" : "10G", "cpus-per-task" : num_threads, "time" : "30","mail-type" : "FAIL,INVALID_DEPEND", "mail-user" : "patrick.bohn@helmholtz-hiri.de"})

minq = "33"

in_dir = f"{data_folder}/fastq"
out_dir = f"{data_folder}/fastq_minq{minq}"

for sample in samples:
    R1 = f"{in_dir}/{sample}_1.fq.gz"
    R2 = f"{in_dir}/{sample}_2.fq.gz"
    O1 = f"{out_dir}/{sample}_1.fq.gz"
    O2 = f"{out_dir}/{sample}_2.fq.gz"
    
    fqi = f"{data_folder}/fastq_stats/{sample}"
    command = f"python3 {os.getcwd()}/functions/filter_fastq.py -p -i1 {R1} -i2 {R2} -o1 {O1} -o2 {O2} -fqi {fqi} -m {minq}"
    #print(command)
    slurm.run(command)

after filtering one has to re-align with bowtie (see 1_Illumina-DMS-MaP_workflow)

# Empricial mutation rates (perbase)

Calculate empirical mean error rate (from bam files generated with LAST (Nanopore) or bowtie (Illumina)

In [ ]:
#calculate for in cell samples only
samples = [sample for sample in samples if "cell" in sample]

In [ ]:
#specify path to perbase binary (we used v 0.8.5) https://github.com/sstadick/perbase/releases/tag/v0.8.5 
perbase = "..."


In [ ]:
#note: includes per position Q22 filter (if below Q22, it is counted as N instead - will still count to total DEPTH)
from slurmpy import Slurm

pids = {}
job_name = "perbase"
num_threads = 10
s = Slurm(job_name, {"partition" : "cpu", "mem" : "10G", "cpus-per-task" : num_threads, "time" : "30",  "mail-user" : "patrick.bohn@helmholtz-hiri.de"})
pids[job_name] = {}
os.makedirs("data/perbase", exist_ok=True)

reference_fasta = f"{data_folder}/references/transcripts_PCR1/RT1_unspliced1.fa"

for sample in samples:
    os.makedirs(f"{data_folder}/perbase/{sample}/", exist_ok=True)
    output_path = f"{data_folder}/perbase/{sample}/RT1_unspliced1.txt.gz"
    
    #specify BAM files from bowtie
    BAM_file =  f"{data_folder}/bam/{sample}/RT1_unspliced1/{sample}.bam"

    command = f"""
    {perbase} base-depth -Q 22 -t {num_threads} -r {reference_fasta} {BAM_file} | gzip > {output_path}
    """
    pids[job_name][sample] = s.run(command)

In [ ]:
import pandas as pd
tmp_data = []

for sample in samples:
    output_path = f"data/perbase/{sample}/RT1_unspliced1.txt.gz"
    try:
        test_df = pd.read_csv(output_path, sep="\t")
    except:
        print("could not read in", output_path)
    test_df["sample"] = sample
    tmp_data.append(test_df)

perbase_df = pd.concat(tmp_data)
print("Total number of nt read in:", perbase_df["DEPTH"].sum())

In [ ]:
#remove N counts from total number (to calculate percentage)
perbase_df["DEPTH"] = perbase_df["DEPTH"] - perbase_df["N"]

#pivot df so that we can calculate % values easily
tmp_df = pd.melt(perbase_df, id_vars=["REF", "POS", "REF_BASE", "DEPTH", "NEAR_MAX_DEPTH", "sample"], value_vars =["A", "C", "G", "T", "N", "INS", "DEL", "REF_SKIP", "FAIL"], value_name="count")
pivot_df = tmp_df[tmp_df["variable"] != "N"].copy()

pivot_df["percent"] = 100*pivot_df["count"]/pivot_df["DEPTH"]
pivot_df.loc[pivot_df["DEPTH"] <1, "percent"] = np.nan

#convert all ref bases to upper to fix grouping
pivot_df["REF_BASE"] = pivot_df["REF_BASE"].str.upper()

#new column to easily filter out correct basecalls
pivot_df["match"] = pivot_df["REF_BASE"] == pivot_df["variable"]


In [ ]:
#extract information from samples as separate columns for plotting; assumes sample as "{replicate}_{RT_primer}_{DMS_conc}_{localization}"

pivot_df["replicate"] = pivot_df["sample"].apply(lambda x: x.split("_")[0])
pivot_df["conc"] = pivot_df["sample"].apply(lambda x: x.split("_")[2])

In [ ]:
#ensures samples are always plotted in the same order when specifying as category_orders in plotly
order_dict = {"conc" : ["0mM", "8mM", "17mM", "34mM", "57mM", "85mM"], 
              "replicate" : ["Rep1", "Rep2"],
             "REF_BASE" : ["A", "C", "G", "T"],
             "variable" : ["A", "C", "G", "T", "INS", "DEL", "REF_SKIP"]}

In [ ]:
fig = px.box(pivot_df[(pivot_df["match"]) & (pivot_df["DEPTH"]>1000)], color="conc",x="conc",  y="percent", category_orders = order_dict, color_discrete_sequence =colorscale)
fig.update_yaxes(range=[80,100], dtick=2)
fig.update_layout(height=400, width=500)
fig.update_traces(marker_size=2)
fig.show(renderer="svg")

In [ ]:
# Generate a new df to plot mismatch rates
mismatch_df = pivot_df[(pivot_df["match"])].copy()
mismatch_df["percent"] = 100- mismatch_df["percent"]

In [ ]:
fig = px.box(mismatch_df[mismatch_df["DEPTH"]>1000], color="conc",x="conc",  y="percent", category_orders = order_dict, color_discrete_sequence =colorscale)
fig.update_yaxes(range=[0,15], dtick=1)
fig.update_layout(height=400, width=500)
fig.update_traces(marker_size=2)
fig.write_image(f"figures/illumina_mut_rate_box.svg")
fig.show(renderer="svg")

In [ ]:
fig = px.box(mismatch_df[mismatch_df["DEPTH"]>1000], facet_col="REF_BASE", color="conc",x="conc",  y="percent", category_orders = order_dict, color_discrete_sequence =colorscale)
fig.update_yaxes(range=[0,15], dtick=1)
fig.update_layout(height=400, width=800)
fig.update_traces(marker_size=2)
fig.write_image(f"figures/illumina_mut_rate_per_nt_box.svg")

fig.show(renderer="svg")

In [ ]:
fig = px.box(pivot_df[~pivot_df["match"]], facet_col="REF_BASE", color="variable", x="conc", y="percent_minus_control", category_orders = order_dict)
fig.update_traces(marker_size=1)
fig.update_yaxes(range = [-0.1,5])
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(height=400, width=1200)

In [ ]:
# calc mean and plot line chart

In [ ]:
mean_df = pivot_df.groupby(["conc", "REF_BASE", "variable"]).agg({"percent" : ["mean"]}).reset_index()
mean_df.columns = ["conc", "REF_BASE", "variable", "percent"]

mean_df["conc"] = pd.Categorical(mean_df["conc"], categories = order_dict["conc"], ordered=True)
mean_df.sort_values(by="conc", inplace=True)

mean_df["conc"] = mean_df["conc"].apply(lambda x: int(x.split("mM")[0]))

In [ ]:
fig = px.line(mean_df, x="conc", y="percent", color="variable", facet_col="REF_BASE", category_orders = order_dict)
fig.update_yaxes(range=[0,2], dtick=0.2)
fig.update_xaxes(type="category")
fig.update_layout(height=400, width=800)
#fig.write_image(f"figures/illumina_comparison/mut_rates/{seq_platform}_mut_type_per_nt_line.svg")
fig.show()

In [ ]:
fig = px.line(mean_df, x="conc", y="percent", color="variable", facet_col="REF_BASE", category_orders = order_dict)
fig.update_yaxes(type="log", range=[-2.5,0.6])
fig.update_xaxes(type="category")
fig.update_layout(height=400, width=800)
fig.write_image(f"figures/illumina_mut_type_per_nt_log_line.svg")
fig.show()

In [ ]:
mean_df["match"] = mean_df["REF_BASE"] == mean_df["variable"]
mean_df["conc"] = mean_df["conc"].apply(lambda x: str(x) + "mM")

In [ ]:
fig = px.pie(mean_df[~mean_df["match"]], values='percent', names='variable', facet_col="REF_BASE", facet_row="conc", category_orders = order_dict)
fig.update_layout(height=1300, width=1000)
fig.write_image(f"figures/illumina_mut_type_per_nt_pie.svg")
fig.show()